In [444]:
# Imports
import random
import numpy as np


In [445]:
"""
Payout Matrix
C - cooperate
D - defect

(x / y) => (player1 years of freedom / player2 years of freedom)
"""
payout = {
    ("C", "C"): (3, 3),
    ("C", "D"): (0, 5),
    ("D", "C"): (5, 0),
    ("D", "D"): (1, 1),
}


In [446]:
"""
Global variables and utility functions
"""

cooperate = "C"
defect = "D"
cooperate_or_deflect = ["C", "D"]

# implementation of switch to get scores because old python :)
# def get_scores(player1_move, player2_move):
#     return tuple(map(int, payout.get((player1_move, player2_move), (0, 0))))



# converting score to years of freedom
def convert_scores_to_years_of_freedom(scores_):
    return [(score, ) for score in scores_]



In [447]:
"""
Implementation of strategies
"""

# win-stay strategy
def win_stay(last_move, my_history, enemy_history, my_score, enemy_score):
    if len(my_history) == 0:
        return random.choice(cooperate_or_deflect)
    elif enemy_history[-1] == cooperate:
        return last_move
    else:
        return random.choice(cooperate_or_deflect)


# lose-shift strategy
def lose_shift(last_move, my_history, enemy_history, my_score, enemy_score):
    if len(my_history) == 0:
        return random.choice(cooperate_or_deflect)
    elif enemy_history[-1] == defect:
        return last_move
    else:
        return defect


# tit-for-tat strategy
def tit_for_tat(last_move, my_history, enemy_history, my_score, enemy_score):
    if len(my_history) == 0:
        return cooperate
    else:
        return enemy_history[-1]


# start with cooperating, in the half game start defecting
def fancy_betrayer(last_move, my_history, enemy_history, my_score, enemy_score):
    if len(my_history) < len(enemy_history) / 2:
        return cooperate
    else:
        return defect


def mirroring(last_move, my_history, enemy_history, my_score, enemy_score):
    if len(my_history) == 0:
        return random.choice(cooperate_or_deflect)

    my_last_move = my_history[-1]
    enemy_last_move = enemy_history[-1]

    if my_last_move == enemy_last_move:
        return enemy_last_move
    else:
        return my_last_move


In [448]:
"""
Implementation of playing single match and tournament
"""

# playing a single match
def play_match(player1, player2, num_rounds_):
    player1_score = 0
    player2_score = 0
    player1_history = []
    player2_history = []

    for round_ in range(num_rounds):
        # player move
        # player1_move = player1(player2_history[-1] if len(player2_history) > 0 else None, player1_history, player2_history, player1_score, player2_score)
        # player2_move = player2(player1_history[-1] if len(player1_history) > 0 else None, player2_history, player1_history, player2_score, player1_score)
        player1_move = player1(player2_history[-1] if len(player2_history) > 0 else None, player1_history, player2_history, player1_score, player2_score)
        player2_move = player2(player1_history[-1] if len(player1_history) > 0 else None, player2_history, player1_history, player2_score, player1_score)


        # get scores
        if (player1_move, player2_move) in payout:
            player1_score += payout[(player1_move, player2_move)][0]
            player2_score += payout[(player1_move, player2_move)][1]

        # update player's history
        player1_history.append(player1_move)
        player2_history.append(player2_move)

    return player1_score, player2_score

# playing tournament
def play_tournament(strategies_, num_rounds_):
    scores_ = {}

    for strategy in strategies_:
        scores_[strategy.__name__] = 0

    for i in range(len(strategies_)):
        for j in range(i + 1, len(strategies_)):
            player1_score, player2_score = play_match(strategies_[i], strategies_[j], num_rounds_=num_rounds_)
            scores_[strategies_[i].__name__] += player1_score
            scores_[strategies_[j].__name__] += player2_score

    return scores_



In [449]:
"""
implementation
"""

strategies = [win_stay, lose_shift, tit_for_tat, fancy_betrayer, mirroring]
num_rounds = 200

scores = play_tournament(strategies, num_rounds_=num_rounds)
years_of_freedom = convert_scores_to_years_of_freedom(list(scores.values()))

player_years_of_freedom = []
for strategy in strategies:
    years_of_freedom_sum = sum(years_of_freedom[i][0] for i in range(len(years_of_freedom)) if strategy.__name__ == list(scores.keys())[i])
    player_years_of_freedom.append((strategy.__name__, years_of_freedom_sum))

player_years_of_freedom_sorted = sorted(player_years_of_freedom, key=lambda x: x[1], reverse=True)

for name, years in player_years_of_freedom_sorted:
    print(name, "scored", years, "years of freedom.")


fancy_betrayer scored 2004 years of freedom.
tit_for_tat scored 1597 years of freedom.
win_stay scored 1414 years of freedom.
mirroring scored 1401 years of freedom.
lose_shift scored 1151 years of freedom.
